# Modelo de Clasificación Nutricional de Platos Bolivianos

Este notebook desarrolla un modelo de aprendizaje supervisado para clasificar platos bolivianos según su perfil nutricional.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importar utilidades
import sys
import os
sys.path.append(os.path.join(os.path.dirname("__file__"), 'utils'))

from cargar_datos import cargar_datos_ingredientes, cargar_datos_nutricion, procesar_ingredientes, combinar_datos
from preprocesamiento import preprocesar_texto_ingredientes, crear_etiquetas_nutricionales, tokenizar_ingredientes, escalar_datos_nutricionales, codificar_etiquetas

# Modelado
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
# Cargar los datasets
df_ingredientes = cargar_datos_ingredientes('../dataset_ingredientes.csv')
df_nutricion = cargar_datos_nutricion('../nutricion/datos_nutricionales.csv')

# Combinar los datasets
df = combinar_datos(df_ingredientes, df_nutricion)
print(f"Forma del dataset combinado: {df.shape}")
print(df.head())

In [ ]:
# Preprocesamiento de texto
df = preprocesar_texto_ingredientes(df)

# Crear etiquetas nutricionales
df = crear_etiquetas_nutricionales(df)

# Mostrar las nuevas columnas
print(df[['plato', 'proteina', 'categoria_proteina', 'carbohidratos', 'categoria_carbohidratos', 'grasa', 'categoria_grasa']].head())

In [ ]:
# Visualización de distribución de categorías
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

sns.countplot(data=df, x='categoria_proteina', ax=axes[0,0])
axes[0,0].set_title('Distribución de Categorías de Proteína')

sns.countplot(data=df, x='categoria_carbohidratos', ax=axes[0,1])
axes[0,1].set_title('Distribución de Categorías de Carbohidratos')

sns.countplot(data=df, x='categoria_grasa', ax=axes[1,0])
axes[1,0].set_title('Distribución de Categorías de Grasa')

sns.countplot(data=df, x='categoria_calorias', ax=axes[1,1])
axes[1,1].set_title('Distribución de Categorías de Calorías')

plt.tight_layout()
plt.show()

In [ ]:
# Tokenizar ingredientes
tokenizer, sequences = tokenizar_ingredientes(df)
print(f"Forma de las secuencias tokenizadas: {sequences.shape}")
print(f"Número de palabras únicas: {len(tokenizer.word_index)}")

In [ ]:
# Separar características y etiquetas para clasificación de proteínas
X = sequences
y, label_encoder = codificar_etiquetas(df, 'categoria_proteina')
y = y['categoria_proteina_codificado'].values

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamaños - Entrenamiento: {X_train.shape[0]}, Prueba: {X_test.shape[0]}")

In [ ]:
# Crear y entrenar modelo de red neuronal
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50
max_length = X.shape[1]
num_classes = len(np.unique(y))

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64, dropout=0.5, recurrent_dropout=0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

In [ ]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)

# Evaluar el modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Precisión en datos de prueba: {test_accuracy:.4f}")

In [ ]:
# Visualizar métricas del entrenamiento
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Precisión Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión Validación')
plt.title('Precisión del Modelo')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Pérdida Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida Validación')
plt.title('Pérdida del Modelo')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Predicciones y métricas de evaluación
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Reporte de clasificación
target_names = label_encoder.classes_
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred_classes, target_names=target_names))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.title('Matriz de Confusión')
plt.xlabel('Predicho')
plt.ylabel('Real')
plt.show()

In [ ]:
# Probar con un ejemplo
def predecir_categoria_proteina(ingredientes: str, model, tokenizer, label_encoder, max_length):
    # Procesar ingredientes
    ingredientes_lista = procesar_ingredientes(ingredientes)
    texto_ingredientes = ' '.join(ingredientes_lista)
    
    # Tokenizar
    sequence = tokenizer.texts_to_sequences([texto_ingredientes])
    sequence = pad_sequences(sequence, maxlen=max_length, padding='post')
    
    # Predecir
    prediccion = model.predict(sequence)
    clase_predicha = np.argmax(prediccion, axis=1)[0]
    categoria_predicha = label_encoder.inverse_transform([clase_predicha])[0]
    
    return categoria_predicha

# Ejemplo de uso
ingredientes_ejemplo = "['carne de res', 'papa', 'cebolla', 'tomate', 'ají']"
categoria_predicha = predecir_categoria_proteina(ingredientes_ejemplo, model, tokenizer, label_encoder, max_length)
print(f"Categoría de proteína predicha para {ingredientes_ejemplo}: {categoria_predicha}")